In [24]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.utils import class_weight

from collections import Counter

from sklearn.metrics import accuracy_score
from sklearn.preprocessing import Imputer
from sklearn.metrics import log_loss, accuracy_score
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.ensemble import RandomForestClassifier
from IPython.display import display, HTML
from sklearn.externals import joblib
from sklearn.cross_validation import cross_val_score
from sklearn.metrics import f1_score, make_scorer
#import graphviz
from pprint import pprint
from sklearn.learning_curve import validation_curve
from sklearn.cross_validation import train_test_split
import utils

#utils.load_data()

# reproducibility
seed = 123

%matplotlib inline
input_file = "../data/ait_result_dataset.V37.csv"
df = pd.read_csv(input_file, header = 0)
classes= np.unique(df['target'])

print("DF original: "+str(df.shape))
#df=df.replace('NaN',np.nan)
df=df.dropna(axis=1, how='all')
#df=df.dropna(axis=1, thresh=len(df) - 10)
#df=df.dropna(axis=0, how='all')
print("DF after dropna: "+str(df.shape))
df=df.replace(np.nan,0)

df=df.replace('N',0)
df=df.replace('O',1)
df=df.replace('A',2)
df=df.replace('~',3)
df[['target']] = df[['target']].astype(int)
target = df['target']

df=df.drop(['target'],axis=1)
#display(df)
X = df.as_matrix()
#n_features=numpy_array.shape[1]
#X=numpy_array[:,0:n_features-1]
#y=numpy_array[:,n_features-1]

#display(pd.DataFrame(X))
#display(pd.DataFrame(y))

#print("X:"+str(X.shape))
#imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
#imp.fit(X)
#X = imp.transform(X)
#print("X after imputer:"+str(X.shape))

#display(pd.DataFrame(X))


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)



d = xgb.DMatrix(df.values, target.values)



print("Train label distribution:")
print(Counter(y_train))

print("\nTest label distribution:")
print(Counter(y_test))
#display(df)
print("\nTrain dataset contains {0} rows and {1} columns".format(X_train.shape[0], X_train.shape[1]))
print("Test dataset contains {0} rows and {1} columns".format(X_test.shape[0], X_test.shape[1]))

DF original: (8528, 387)
DF after dropna: (8528, 387)
Train label distribution:
Counter({0.0: 4070, 1.0: 1929, 2.0: 594, 3.0: 229})

Test label distribution:
Counter({0.0: 1006, 1.0: 486, 2.0: 164, 3.0: 50})

Train dataset contains 6822 rows and 386 columns
Test dataset contains 1706 rows and 386 columns


In [25]:
params = {
    'objective': 'multi:softmax',
    'max_depth': 7,
    'learning_rate': 0.26046515748913901,
    'silent': 1,
    'n_estimators': 110,
    'nthread': 4,
    'colsample_bytree': 0.81958831684028921,
    'gamma':0.25,
    'subsample':0.93168572417786366,
    'min_child_weight':0.9,
    'colsample_bylevel':1,
    'max_delta_step':0,
    'missing':None,
    'reg_alpha':0,
    'scale_pos_weight':1,
    'num_class':4
}
    


#{'colsample_bytree': 0.18032342312495409, 'gamma': 0, 'learning_rate': 0.13351271673674348, 'max_depth': 4, 'min_child_w
#eight': 0.9, 'n_estimators': 200, 'subsample': 0.71772272477228749}
#84.42 5-fold CV
 
#XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
#       colsample_bytree=0.27958200096140395, gamma=0.25,
#       learning_rate=0.10836619673151071, max_delta_step=0, max_depth=9,
#       min_child_weight=1, missing=None, n_estimators=200, n_jobs=1,
#       nthread=4, objective='multi:softprob', random_state=0, reg_alpha=0,
#       reg_lambda=1, scale_pos_weight=1, seed=1234, silent=1,
#       subsample=0.93166895175080411)
#{'colsample_bytree': 0.27958200096140395, 'gamma': 0.25, 'learning_rate': 0.10836619673151071, 'max_depth': 9, 'min_child_weight': 1, 'n_estimators': 200, 'subsample': 0.93166895175080411}
#0.8452020226674676 5-fold-cv


#XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
#       colsample_bytree=0.27664819563062615, gamma=0.5,
#       learning_rate=0.075965443835642077, max_delta_step=0, max_depth=9,
#       min_child_weight=0.9, missing=None, n_estimators=200, n_jobs=1,
#       nthread=4, objective='multi:softprob', random_state=0, reg_alpha=0,
#       reg_lambda=1, scale_pos_weight=1, seed=1234, silent=1,
#       subsample=0.81186483944302257)
#{'colsample_bytree': 0.27664819563062615, 'gamma': 0.5, 'learning_rate': 0.075965443835642077, 'max_depth': 9, 'min_chil
#d_weight': 0.9, 'n_estimators': 200, 'subsample': 0.81186483944302257}
#0.845377783528895
#PS D:\cinc2017>


 
num_rounds = 100

watchlist  = [(dtrain,'train')]

def misclassified(pred_probs, dtrain):
    return 'misclassified', np.sum(dtrain.get_label() != pred_probs > 0.5)

def f1(pred_probs, dtrain):
    return 'f1',f1_score(dtrain.get_label(), pred_probs,labels=[0,1,2],average='macro')


#
bst = xgb.train(params, dtrain, num_rounds,watchlist,feval=f1, maximize=False)
#hist = xgb.cv(params, dtrain, num_rounds, nfold=5, metrics={'merror'}, seed=seed)

bst

#joblib.dump(bst, 'xgboost.pkl', compress = 1)

#xgb.plot_importance(bst)
#xgb.to_graphviz(bst,  num_trees=2)
#y_test_preds=bst.predict(dtest)
#display(pd.DataFrame(y))

#pd.crosstab(
#    pd.Series(y_test, name='Actual'),
#    pd.Series(y_test_preds, name='Predicted'),
#    margins=True
#)

#print("F1: {:1.4f}".format(f1_score(y_test, y_test_preds,labels=[0,1,2],average='macro'))) 

[0]	train-merror:0.099203	train-f1:0.890085
[1]	train-merror:0.081027	train-f1:0.911334
[2]	train-merror:0.073288	train-f1:0.919672
[3]	train-merror:0.068363	train-f1:0.926082
[4]	train-merror:0.062969	train-f1:0.933065
[5]	train-merror:0.059803	train-f1:0.93675
[6]	train-merror:0.056285	train-f1:0.940346
[7]	train-merror:0.053236	train-f1:0.943766
[8]	train-merror:0.048663	train-f1:0.949092
[9]	train-merror:0.045263	train-f1:0.952694
[10]	train-merror:0.040689	train-f1:0.958146
[11]	train-merror:0.037875	train-f1:0.961576
[12]	train-merror:0.033185	train-f1:0.965876
[13]	train-merror:0.029784	train-f1:0.969616
[14]	train-merror:0.027439	train-f1:0.972231
[15]	train-merror:0.025211	train-f1:0.974753
[16]	train-merror:0.022162	train-f1:0.978186
[17]	train-merror:0.021459	train-f1:0.978636
[18]	train-merror:0.019348	train-f1:0.98126
[19]	train-merror:0.018175	train-f1:0.982797
[20]	train-merror:0.016065	train-f1:0.9846
[21]	train-merror:0.014306	train-f1:0.986537
[22]	train-merror:0.0132

In [26]:
y_pred=bst.predict(d, pred_contribs=True)

In [9]:
y_pred

array([[  6.57878583e-03,  -2.73995707e-03,  -1.70348398e-02, ...,
          7.14087160e-04,  -5.89223564e-01,  -1.54275155e+00],
       [  1.68673415e-02,   7.68226013e-03,  -6.16669469e-03, ...,
         -3.62236169e-03,  -3.37405086e-01,  -1.54275155e+00],
       [  3.42118852e-02,  -1.41881863e-02,  -1.14272372e-03, ...,
          6.81905192e-04,  -6.03545904e-01,  -1.54275155e+00],
       ..., 
       [  2.00909749e-03,  -4.33204230e-03,  -2.75124470e-03, ...,
          4.18828771e-04,  -6.60088956e-01,  -1.54275155e+00],
       [  2.70310901e-02,   1.75513793e-02,  -2.48992490e-03, ...,
         -4.89302725e-03,  -3.63840789e-01,  -1.54275155e+00],
       [ -1.59238391e-02,  -1.03100901e-02,  -1.02544855e-02, ...,
          5.08764060e-04,  -6.21760488e-01,  -1.54275155e+00]], dtype=float32)

In [38]:
import shap

# load JS visualization code to notebook
shap.initjs() 
shap.visualize(y_pred[0,:], feature_names=df.columns, data=X[0,:])

In [40]:
import lime


In [41]:
display(pd.DataFrame(y_pred))

,0,1,2,3,4,5,6,7,8,9,...,1538,1539,1540,1541,1542,1543,1544,1545,1546,1547
0,0.006579,-0.002740,-0.017035,-0.018867,0.000351,-0.002586,-0.047367,0.023176,0.007471,-0.002587,...,0,0,-0.035462,-0.001293,-0.017870,-0.000027,-0.577838,0.000714,-0.589224,-1.542752
1,0.016867,0.007682,-0.006167,0.031200,0.008425,0.026501,0.030695,0.016185,-0.000431,0.064111,...,0,0,0.084297,-0.001259,-0.034979,-0.000925,0.202211,-0.003622,-0.337405,-1.542752
2,0.034212,-0.014188,-0.001143,0.009550,0.009794,0.008780,0.026867,0.033042,0.004458,0.029233,...,0,0,-0.037313,-0.001293,-0.016109,0.000037,-0.648612,0.000682,-0.603546,-1.542752
3,-0.020308,0.009876,-0.002705,-0.007755,-0.015449,-0.008891,-0.082614,-0.052836,-0.006180,0.009183,...,0,0,-0.045936,-0.001156,-0.017313,-0.000554,-0.504275,-0.002812,0.100976,-1.542752
4,-0.011742,0.015012,-0.001787,0.005897,-0.046160,-0.009215,-0.000700,0.036136,0.001416,0.058082,...,0,0,-0.024623,-0.001769,-0.019403,-0.000203,0.231406,-0.004644,-0.626536,-1.542752
5,0.007698,0.036356,-0.002676,0.012470,0.018426,0.036774,-0.022293,0.022086,-0.000392,0.012901,...,0,0,-0.035303,-0.001769,-0.017649,-0.000437,-0.534224,0.000497,-0.548558,-1.542752
6,-0.019622,-0.001968,-0.002522,-0.015567,-0.010108,0.014054,-0.063677,0.048371,-0.006262,0.082831,...,0,0,-0.037182,-0.001769,-0.017969,0.000064,-0.528306,0.000656,-0.573415,-1.542752
7,-0.004126,-0.001528,-0.003403,0.006330,-0.001154,0.010849,-0.147130,-0.031968,0.007933,0.011468,...,0,0,-0.038158,-0.001293,-0.017049,0.000060,-0.571431,0.000729,-0.610243,-1.542752
8,0.002510,-0.004546,-0.002228,-0.003407,-0.004891,-0.004357,0.008240,-0.001323,0.004534,-0.001671,...,0,0,-0.054978,-0.001769,-0.016895,-0.000243,-0.659065,-0.003997,-0.674875,-1.542752
9,0.029871,-0.004412,-0.001996,0.033049,-0.008043,-0.024411,-0.043743,0.024944,-0.003145,0.059001,...,0,0,-0.039282,-0.000694,-0.017870,-0.000027,-0.594564,0.000656,-0.578205,-1.542752


In [60]:
import lime.lime_tabular

explainer = lime.lime_tabular.LimeTabularExplainer(X,
                                                   feature_names=list(df.columns),
                                                   class_names=classes,
                                                   discretize_continuous=False)



rf = sklearn.ensemble.RandomForestClassifier(n_estimators=100)
rf.fit(X, y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [69]:
i = np.random.randint(0, X.shape[0])
display(i)
exp = explainer.explain_instance(X[i,:], rf.predict_proba, num_features=10, top_labels=2)
exp.show_in_notebook(show_table=True, show_all=False)


4799

In [65]:
rf.predict_proba

<bound method ForestClassifier.predict_proba of RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)>

In [67]:
bst.predict_proba

AttributeError: 'Booster' object has no attribute 'predict_proba'